In [2]:
import pandas as pd
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()  # loads .env file
file_path = os.getenv('FILE_PATH')

In [6]:
df = pd.read_excel(file_path)

In [7]:
df.columns

Index(['Row ID', 'Order Priority', 'Discount', 'Unit Price', 'Shipping Cost',
       'Customer ID', 'Customer Name', 'Ship Mode', 'Customer Segment',
       'Product Category', 'Product Sub-Category', 'Product Container',
       'Product Name', 'Product Base Margin', 'Country', 'Region',
       'State or Province', 'City', 'Postal Code', 'Order Date', 'Ship Date',
       'Profit', 'Quantity ordered new', 'Sales', 'Order ID'],
      dtype='object')

In [29]:
df = df[df['Profit'] > 0]

In [34]:
product_total = (
    df.groupby('Product Name', as_index=False)['Profit']
      .sum()
      .nlargest(10, 'Profit')
      .rename(columns={'Profit': 'TotalProfit'})
)

top_products = product_total['Product Name']

filtered = df[df['Product Name'].isin(top_products)]

# Group by product + state
grouped = (
    filtered.groupby(['Product Name', 'State or Province'], as_index=False)['Profit']
            .sum()
            .rename(columns={'Profit': 'StateProfit'})
)

# Get top state per product (using idxmax)
idx = grouped.groupby('Product Name')['StateProfit'].idxmax()
top_state = grouped.loc[idx].reset_index(drop=True)

# Merge with total product profit
final = top_state.merge(product_total, on='Product Name')

# Calculate percentage
final['Percentage'] = (final['StateProfit'] / final['TotalProfit'] * 100).round(2)

# Reorder columns
final = final[['Product Name', 'TotalProfit', 'State or Province', 'StateProfit', 'Percentage']]

print(final.to_string())




                                                                  Product Name  TotalProfit     State or Province  StateProfit  Percentage
0                                           Canon PC1060 Personal Laser Copier   9607.76838                 Maine    7024.2069       73.11
1  Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind  23865.90420  District of Columbia    9228.2256       38.67
2                                   GBC DocuBind TL300 Electric Binding System  11004.45120                  Ohio    7402.3200       67.27
3                               Global Troy™ Executive Leather Low-Back Tilter  18628.14530            California    9289.4953       49.87
4                                         Hewlett Packard LaserJet 3310 Copier   8798.18310              New York    8798.1831      100.00
5          Hewlett-Packard Business Color Inkjet 3000 [N, DTN] Series Printers   7576.11000                Oregon    7576.1100      100.00
6                          